In [1]:
import rasterio
import numpy as np
from rasterio import windows
from itertools import product

# Window loading

In [25]:
def window_overlap(src, overlap, width, height, boundless=True):
    offsets = product(range(0, src.meta['width'], width), range(0, src.meta['height'], height))
    full_window = windows.Window(col_off=0, row_off=0, width=src.meta['width'], height=src.meta['height'])
    for col_off, row_off in offsets:
        window = windows.Window(
            col_off=col_off - overlap,
            row_off=row_off - overlap,
            width=width,
            height=height)
        
        if boundless:
            yield window
        else:
            yield window.intersection(full_window)


In [51]:
%%time
# use composite image as test tile
path_s2 = "../../data/TestDataSet/S2_comp_first.tif"

width = 32
height = 32
overlap = 4

ls_stamp = []
# %load_ext memory_profiler
with rasterio.open('../../data/TestDataSet/S2_comp_first.tif') as src:
    for w in window_overlap(src, overlap, width, height):
        ls_stamp.append(src.read([1,2,3], window=w, boundless=True))
stamp = np.stack(ls_stamp)

CPU times: user 7min 53s, sys: 12.2 s, total: 8min 5s
Wall time: 12min 9s


In [53]:
stamp.shape

(97969, 3, 32, 32)